In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import nltk
import math

In [2]:
def conv_to_char(n):
    s = np.array(list(n))
    st = ""
    for i in s:
        st += chr(i)
    return st

In [3]:
def features_of_images_train_val(location_folder):

    list_of_files = os.listdir(location_folder)

    full_file_path = np.array([])
    for i in list_of_files:
        full_file_path = np.append(full_file_path,location_folder+i)

    id = np.array([])
    cls = np.array([])
    image = np.array([])
    ctr = 0
    for i in full_file_path:
        raw_img = tf.data.TFRecordDataset(i)
        image_description = {
            "id" : tf.io.FixedLenFeature([],tf.string),
            "class": tf.io.FixedLenFeature([],tf.int64),
            "image": tf.io.FixedLenFeature([],tf.string)
        }
        y = lambda x: tf.io.parse_single_example(x,image_description)
        img = raw_img.map(y)

        begin = i.rindex("-")+1
        end = i.index(".tfrec")
        num = int(i[begin:end])

        for dr in img.take(num):
            id = np.append(id,conv_to_char(dr["id"].numpy()))
            cls = np.append(cls,int(dr["class"].numpy()))
            image = np.append(image,tf.io.decode_image(dr["image"].numpy(),dtype="float64"))
            print(ctr)
            ctr += 1
    return id,cls,image

In [4]:
def features_of_images_test(location_folder):
    list_of_files = os.listdir(location_folder)
    
    full_file_path = np.array([])
    for i in list_of_files:
        full_file_path = np.append(full_file_path,location_folder+i)

    id = np.array([])
    image = np.array([])

    for i in full_file_path:
        raw_img = tf.data.TFRecordDataset(i)
        image_description = {
            "id" : tf.io.FixedLenFeature([],tf.string),
            "class": tf.io.FixedLenFeature([],tf.int64),
            "image": tf.io.FixedLenFeature([],tf.string)
        }
        y = lambda x: tf.io.parse_single_example(x,image_description)
        img = raw_img.map(y)

        begin = i.rindex("-")+1
        end = i.index(".tfrec")
        num = int(i[begin:end])

        for dr in img.take(num):
            id = np.append(id,conv_to_char(dr["id"].numpy()))
            image = np.append(image,tf.io.decode_image(dr["image"].numpy(),dtype="float64"))

    return id,image

In [5]:
p = np.array([])
p = np.append(p,4)
print(p)
p = np.append(p,5)
print(p)

[4.]
[4. 5.]


In [6]:
location_folder_train_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train/"

location_folder_train_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/train/"

location_folder_train_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/train/"

location_folder_train_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/train/"

In [7]:
location_folder_val_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/val/"

location_folder_val_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/val/"

location_folder_val_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/val/"

location_folder_val_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/val/"

In [8]:
location_folder_test_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/test/"

location_folder_test_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/test/"

location_folder_test_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/test/"

location_folder_test_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/test/"

### Training Dataset

In [ ]:
id_train_192,cls_train_192,image_train_192 = features_of_images_train_val(location_folder_train_192)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [60]:
id_train_224,cls_train_224,image_train_224 = features_of_images_train_val(location_folder_train_224)

In [62]:
id_train_331,cls_train_331,image_train_331 = features_of_images_train_val(location_folder_train_331)

In [63]:
id_train_512,cls_train_512,image_train_512 = features_of_images_train_val(location_folder_train_512)

In [64]:
len(image_train_192)+len(id_train_224)+len(id_train_331)+len(id_train_512)

0

### Validation Dataset

In [ ]:
id_val_192,cls_val_192,image_val_192 = features_of_images_train_val(location_folder_val_192)

In [ ]:
id_val_224,cls_val_224,image_val_224 = features_of_images_train_val(location_folder_val_224)

In [ ]:
id_val_331,cls_val_331,image_val_331 = features_of_images_train_val(location_folder_val_331)

In [ ]:
id_val_512,cls_val_512,image_val_512 = features_of_images_train_val(location_folder_val_512)

### Testing Dataset

In [ ]:
id_test_192,image_test_192 = features_of_images_test(location_folder_test_192)

In [ ]:
id_test_224,image_test_224 = features_of_images_test(location_folder_test_224)

In [ ]:
id_test_331,image_test_331 = features_of_images_test(location_folder_test_331)

In [ ]:
id_test_512,image_test_512 = features_of_images_test(location_folder_test_512)

### Deep Learning

In [44]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Resizing(height=512,width=512),
    tf.keras.layers.Conv2D(filters=64,kernel_size=4,activation="relu",input_shape=(512,512,3)),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Conv2D(filters=128,kernel_size=8,activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Conv2D(filters=256,kernel_size=16,activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Conv2D(filters=128,kernel_size=8,activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Conv2D(filters=64,kernel_size=4,activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(units=128,activation="relu"),
    tf.keras.layers.Dense(units=104,activation="softmax")
])

In [45]:
model.compile(
    optimizer=tf.keras.optimizers.legacy.RMSprop(),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=tf.keras.metrics.categorical_accuracy,
)

In [46]:
model.fit(
    x=image_train_192,
    y=cls_train_192,
    batch_size=10,
    epochs=100,
    verbose=2,
    callbacks=tf.keras.callbacks.EarlyStopping(patience=2)
)

Epoch 1/100


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_t